# MNIST Autoencoder & Classifier

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset

def load_mnist(transform=transforms.ToTensor()):
    # MNIST dataset
    train_set = dsets.MNIST(root='data/',
                            train=True,
                            transform=transform,
                            download=True
                            )
    valid_set = dsets.MNIST(root='data/',
                            train=False,
                            transform=transform,
                            download=True
                            )

    train_data = train_set.train_data.float() / 255.
    train_labels = train_set.train_labels
    test_data = valid_set.test_data.float() / 255.
    test_labels = valid_set.test_labels
    
    train_data = train_data.view(len(train_data), -1)
    test_data = test_data.view(len(test_data), -1)
    
    return train_data, train_labels, test_data, test_labels

## Autoencoder

In [3]:
class AutoEncoder(nn.Module):
    
    def __init__(self, input_dim, btl_size, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()
        
        self.enc1 = nn.Linear(input_dim, 400)
        self.enc2 = nn.Linear(400, btl_size)
        self.dec1 = nn.Linear(btl_size, 400)
        self.dec2 = nn.Linear(400, output_dim)
        self.act = nn.LeakyReLU()
        self.final_act = nn.Sigmoid()
        
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        
        z = self.act(self.enc1(x))
        z = self.act(self.enc2(z))
        z = self.act(self.dec1(z))
        y = self.final_act(self.dec2(z))
        
        return y

In [4]:
x, _, test_x, _ = load_mnist()

net = AutoEncoder(28**2, 20, 28**2)
crit = nn.MSELoss()
optim = torch.optim.Adam(net.parameters())

print(net)
n_epochs = 10
batch_size = 128

train = DataLoader(TensorDataset(x, x), 
                   batch_size=batch_size, 
                   shuffle=True, 
                   num_workers=0
                  )

for i in range(n_epochs):
    for j, (x_j, y_j) in enumerate(train):
        net.zero_grad()

        y_hat = net(x_j)
        loss = crit(y_hat, y_j)        

        loss.backward()
        optim.step()
        
    print('epoch: %d\tloss: %.4f' % (i, loss))    

AutoEncoder(
  (enc1): Linear(in_features=784, out_features=400, bias=True)
  (enc2): Linear(in_features=400, out_features=20, bias=True)
  (dec1): Linear(in_features=20, out_features=400, bias=True)
  (dec2): Linear(in_features=400, out_features=784, bias=True)
  (act): LeakyReLU(negative_slope=0.01)
  (final_act): Sigmoid()
)
epoch: 0	loss: 0.0197
epoch: 1	loss: 0.0162
epoch: 2	loss: 0.0147
epoch: 3	loss: 0.0136
epoch: 4	loss: 0.0119
epoch: 5	loss: 0.0115
epoch: 6	loss: 0.0109
epoch: 7	loss: 0.0107
epoch: 8	loss: 0.0100
epoch: 9	loss: 0.0098


## Classifier

In [5]:
class Classifier(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()
        
        self.h1 = nn.Linear(input_dim, 400)
        self.h2 = nn.Linear(400, 200)
        self.h3 = nn.Linear(200, 100)
        self.h4 = nn.Linear(100, 50)
        self.h5 = nn.Linear(50, 10)
        self.act = nn.LeakyReLU()
        self.final_act = nn.LogSoftmax(dim=-1)
        
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        
        z = self.act(self.h1(x))
        z = self.act(self.h2(z))
        z = self.act(self.h3(z))
        z = self.act(self.h4(z))
        y = self.final_act(self.h5(z))
        
        return y

In [6]:
x, y, test_x, test_y = load_mnist()

net = Classifier(28**2, 10)
crit = nn.NLLLoss()
optim = torch.optim.Adam(net.parameters())

print(net)
n_epochs = 10
batch_size = 128

train = DataLoader(TensorDataset(x, y), 
                   batch_size=batch_size, 
                   shuffle=True, 
                   num_workers=0
                  )

for i in range(n_epochs):
    for j, (x_j, y_j) in enumerate(train):
        net.zero_grad()

        y_hat = net(x_j)
        loss = crit(y_hat, y_j)

        max_idx = y_hat.topk(1)[1].squeeze()
        correct_cnt = float((max_idx == y_j).sum())

        loss.backward()
        optim.step()
        
    print('epoch: %d\tloss: %.4f\taccuracy: %.4f' % (i, loss, correct_cnt / max_idx.size(0)))

Classifier(
  (h1): Linear(in_features=784, out_features=400, bias=True)
  (h2): Linear(in_features=400, out_features=200, bias=True)
  (h3): Linear(in_features=200, out_features=100, bias=True)
  (h4): Linear(in_features=100, out_features=50, bias=True)
  (h5): Linear(in_features=50, out_features=10, bias=True)
  (act): LeakyReLU(negative_slope=0.01)
  (final_act): LogSoftmax()
)
epoch: 0	loss: 0.1676	accuracy: 0.9479
epoch: 1	loss: 0.2522	accuracy: 0.9583
epoch: 2	loss: 0.0252	accuracy: 1.0000
epoch: 3	loss: 0.1499	accuracy: 0.9479
epoch: 4	loss: 0.0507	accuracy: 0.9896
epoch: 5	loss: 0.0202	accuracy: 0.9896
epoch: 6	loss: 0.0293	accuracy: 0.9792
epoch: 7	loss: 0.0604	accuracy: 0.9792
epoch: 8	loss: 0.0213	accuracy: 0.9896
epoch: 9	loss: 0.0419	accuracy: 0.9896
